# March 23 half-scale test for PixelFrame

This notebook is based on: 
Flexural behavior of externally prestressed beams Part 1: Analytical models" Chee Khoon Ng, Kiang Hwee Tan. (2006)
***


## Notations
***
working on it
***

### Variables that are assumed in this calculation
- fc′ :concrete compressive strength [MPa]
- fpe :effective post tensioning stress [MPa] 

### Calculation Notes
- (from the paper)
Since sections are usually under-reinforced, the behavior will govern by the steel yielding. Therefore, the nonlinear behavior of the concrete is neglected.
- Use Ld = Ls (this test only) 
- Eccentricities measured from the neutral axis
- M is the moment in the constant region
- Mg = moment due to the selfweight
- M(x) is the moment equation due to the load
- Units N, mm, MPa

## Setting up packages

In [10]:
using CSV, DataFrames
using UnPack
using Makie, GLMakie 

### Todo
- [x] Plot everything out vs displacement. 
- [ ]

Check every variable that can be checked. (e mid , forces (static) , )


### Include files

In [11]:
include("input_data.jl") ;
include("functions.jl") ;
include("Interpolations.jl") ;

#### Default values

In [12]:
st = 10.0 ;#step size of the force  inputs
P_lb = 0:st:8300 ; #[lb] (This is based on the test data)
P_N  = 4.448*P_lb ;# [N]
P = P_N ;# This depends on what unit you want to use in the calculation.
M = P*Ls/2.0 ;#given M inputs

#### Instantiates empty arrays to store results

In [13]:
con_size = length(P) ;
fps_history = zeros(con_size) ;
dps_history = zeros(con_size) ;
Icr_history = zeros(con_size) ;
Ie_history  = zeros(con_size) ; 
c_history   = zeros(con_size) ;
dis_history = zeros(con_size) ;
dis_dev_history = zeros(con_size) ;
fc_history  = zeros(con_size) ;

#### Assumes

In [14]:
Icr = Itr # Icrack = I transformed (initial I of concrete and steel).
Ie = Itr # Assume Cracked inertia's = I transformed.
fps = fpe # Assume stress in the tendon = effective post tensioning force.
dps = dps0 # Initial depth of the post tensioning tendon.
Ω =  getOmega(Sec) # coefficient.
#we could do Mcr = 0 , becuase we crack at the begining anyway. 
Mcr = getMcr(Mat, Sec, f, Ω)
# Mcr = 0.00001
# Mcr = 10.0


mcre = Aps * fpe * (em + Zb / Atr) + fr * Zb = 3.5328848800667063e6
dmcr = (Aps * em * (em + Zb / Atr) * (mcre - mg)) / (((1 / Ω) * Itr * Ec) / Eps + Aps * (r ^ 2 - (em * Zb) / Atr)) = 162334.89432065873


3.695219774387365e6

#### Initializing values

In [15]:
Ωc = 0
c  = 0
Ac_req  = 0 
Lc = 0
fc = 0.0
δ_mid = 0
δ_dev = 0
fc = 0.0
Mi = 0

0

#### Initializing figures for vizualization

In [16]:
fig1 = Figure(backgroundcolor = RGBf(0.98,0.98,0.98) , resolution = (1000, 700))
ga = fig1[1,1] = GridLayout()
gb = fig1[1,2] = GridLayout()
title_name = [ "dps", "fps", "DisMid", "c", "Inertia(s)", "withTest"]
fig_monitor = Figure(resolution = (1200, 2000))
axis_monitor1 = [Axis(ga[i,1], title = title_name[i]) for i in 1:5]
axis_monitor2 = [Axis(gb[i,1]) for i in 1:2]

2-element Vector{Axis}:
 Axis (0 plots)
 Axis (0 plots)

#### Main program for calculating the deflection and related variables
[This part is based on fig 7 in the paper]